# Library import

In [1]:
import pandas as pd
from scipy.stats import norm 
import joblib
from sklearn.metrics import roc_auc_score
import os
import random
import xgboost
import numpy as np
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import logging
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Data set creation

In [2]:
InputAOI=pd.read_csv('../data/AOI_training.csv.zip')
InputAOI['PinNumber']=InputAOI['PinNumber'].astype('Int64').astype(str)
InputAOI['FigureID2']=InputAOI['FigureID']
InputAOI['ComponentID2']=InputAOI['ComponentID']


#As ComponentID and FigureID are needed in the return file we store a not modified version as FigureID2

InputAOI['Count_Pin']=InputAOI.groupby(["PanelID","FigureID2","ComponentID2"])["PinNumber"].transform('count')
InputAOI['Count_Pin_Figure']=InputAOI.groupby(["PanelID","FigureID2"])["PinNumber"].transform('count')

InputAOI['Target']=InputAOI['OperatorLabel'].astype('category').cat.codes

InputSPI=pd.DataFrame()
for item in range(4):
    InputSPI=pd.concat([InputSPI,pd.read_csv('../data/SPI_training_'+str(item)+'.csv.zip')])
InputAOI['key_spi']=(InputAOI['PanelID'].astype(str)+'_'+InputAOI['FigureID'].astype(str)+'_'+InputAOI['ComponentID'].astype(str))
InputSPI=InputSPI.dropna(subset=['FigureID'])

InputSPI['FigureID2']=InputSPI['FigureID']
InputSPI['ComponentID2']=InputSPI['ComponentID']
InputSPI['key_spi']=(InputSPI['PanelID'].astype(str)+'_'+InputSPI['FigureID'].astype(int,errors='ignore').astype(str)+'_'+InputSPI['ComponentID'].astype(str))
Features_to_bin=['Volume(%)',  'Area(%)', 'OffsetX(%)','OffsetY(%)']


/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
# We create a model for the line of AOI files where no PinNumber is associated to the defect

aoi_outter=InputAOI.loc[InputAOI.PinNumber=='<NA>']
aoi_outter['FigureID_ComponentID']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)).astype("category").cat.codes
aoi_outter['FigureID_ComponentID_PinNumber']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)+'_'+aoi_outter['PinNumber'].astype(str)).astype("category").cat.codes
aoi_outter['FigureID_ComponentID_PinNumber_AOILabel']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)+'_'+aoi_outter['PinNumber'].astype(str)+aoi_outter['AOILabel'].astype(str)).astype("category").cat.codes
aoi_outter['FigureID_ComponentID_AOILabel']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)+"_"+aoi_outter['AOILabel'].astype(str)).astype("category").cat.codes
list_var_to_encode=['FigureID',"ComponentID","PinNumber","AOILabel","FigureID_ComponentID","FigureID_ComponentID_PinNumber","FigureID_ComponentID_PinNumber_AOILabel","FigureID_ComponentID_AOILabel"]

for var in list_var_to_encode:
    aoi_outter[var]=aoi_outter[var].astype('category').cat.codes
    
    
    

/tmp/ipykernel_21583/2689965088.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aoi_outter['FigureID_ComponentID']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)).astype("category").cat.codes
/tmp/ipykernel_21583/2689965088.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aoi_outter['FigureID_ComponentID_PinNumber']=(aoi_outter['FigureID'].astype(str)+'_'+aoi_outter['ComponentID'].astype(str)+'_'+aoi_outter['PinNumber'].astype(str)).astype("category").cat.codes
/tmp/ipy

In [14]:
InputSPI_grouped=InputSPI.groupby(["PanelID","FigureID2","ComponentID2"])[Features_to_bin].mean().reset_index()
aoi_outter=aoi_outter.merge(InputSPI_grouped,how="left",left_on=["PanelID","FigureID2","ComponentID2"],right_on=["PanelID","FigureID2","ComponentID2"])

In [15]:
list_var_to_encode=["AOILabel","ComponentID","FigureID_ComponentID"]

for var in list_var_to_encode:
    aoi_outter[var]=aoi_outter[var].astype('category').cat.codes


In [16]:
Features_to_bin=['Volume(%)',  'Area(%)', 'OffsetX(%)','OffsetY(%)']

In [17]:
Train_df=aoi_outter.reset_index(drop=True)
y_train=Train_df['Target']


encoder=ce.CatBoostEncoder()
X_train=Train_df

AE_train=encoder.fit_transform(X_train[list_var_to_encode].copy(),y_train)
AE_train.columns=[x+"_encoded" for x in AE_train.columns.tolist()]
X_train_final=pd.concat([X_train,AE_train],axis=1)
joblib.dump(encoder,"Cat_boost_step2_outer")

ratio=np.sum(y_train==0)/np.sum(y_train==1)
col_features=Features_to_bin+[x+"_encoded" for x in list_var_to_encode]+["Count_Pin","Count_Pin_Figure"]

X_train_final=X_train_final.replace(np.nan,0)
X_train_final=X_train_final[col_features].to_numpy()

# Optuna hyperparamétrisation

In [18]:

def objective(trial: Trial,X,y) -> float:
    
    joblib.dump(study, 'study.pkl')

    param = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['exact']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        #'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'n_estimators': trial.suggest_categorical('n_estimators', [30,40,50,70,100,150,200]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'nthread' : -1,
        #'feval': trial.suggest_categorical('feval',[f1_eval]),
        #'scale_pos_weight' : trial.suggest_categorical('scale_pos_weight', [ratio])
        

    }
    
    
    model = xgboost.XGBClassifier(**param,feval=f1_eval)
    
    return cross_val_score(model, X_train_final, y_train, cv=5,scoring=make_scorer(f1_score, pos_label=0)).mean()

In [19]:
from sklearn.metrics import f1_score
import numpy as np
from sklearn.metrics import make_scorer

def f1_eval(y_pred, y_true):
    #y_true = dtrain.get_label()
    err = f1_score(y_true,y_pred,pos_label=0)
    return 'f1_err', err

y_train=Train_df['Target']

study = optuna.create_study(direction='maximize')

optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

#study = optuna.create_study(direction='maximize', storage="sqlite:///xgb_optuna_home_credit.db", study_name="optuna_xgb_output_8")
study.optimize(lambda trial : objective(trial,X_train_final,y_train),n_trials= 10)



In [20]:
Best_trial = study.best_trial.params
model = xgboost.XGBClassifier(**Best_trial)
model.fit(X_train_final,y_train)
print(confusion_matrix(y_train, model.predict(X_train_final),labels=[0,1]))
cross_val_score(model, X_train_final, y_train, cv=5,scoring=make_scorer(f1_score, pos_label=0)).mean()

[[ 335  116]
 [   1 8672]]


0.8174692134918484

In [21]:
model = xgboost.XGBClassifier(**Best_trial)
model.fit(X_train_final,y_train)
joblib.dump(model,"xgboost_step2_outter")


['xgboost_step2_outter']

# Cross validation per panel score

In [33]:
list_panel_id=InputAOI.PanelID.unique().tolist()


def cross_validation_panel_id_creation(list_panel_id,n_fold=5):
    Fold_list=[]
    random.shuffle(list_panel_id)
    list_panel_id=joblib.load("list_panel_id")
    len_fold=(len(list_panel_id))//n_fold
    for fold in range(n_fold):
        Fold_list.append(list_panel_id[fold*len_fold:(fold+1)*len_fold])
    return(Fold_list)
Fold_list=cross_validation_panel_id_creation(list_panel_id)
#Final_df=Final_df.loc[Final_df['ComponentID2']!="BC1"]
f1_score_list=[]
list_confusion_matrix=[]
for Test_fold_indice in range(5):
    print(Test_fold_indice)
    Train_indice=[]
    count_indice_fit_pin_training=0
    for indices in range(len(Fold_list)):
        if indices!=Test_fold_indice:
            Train_indice=Train_indice+Fold_list[indices]
        else:
            Test_indices=Fold_list[indices]
            #Train_indice=Train_indice+Fold_list[indices]

    Train_df=aoi_outter.loc[aoi_outter["PanelID"].isin(Train_indice)].reset_index(drop=True)
    Test_df=aoi_outter.loc[aoi_outter["PanelID"].isin(Test_indices)].reset_index(drop=True)
    X_train=Train_df
    X_val=Test_df
    y_train=Train_df['Target']
    y_val=Test_df['Target']
    y_train=Train_df['Target']
    y_train=Train_df['Target']
    y_val=Test_df['Target']

    encoder=ce.CatBoostEncoder()
    

    AE_train=encoder.fit_transform(X_train[list_var_to_encode].copy(),y_train)
    AE_train.columns=[x+"_encoded" for x in AE_train.columns.tolist()]
    X_train_final=pd.concat([X_train,AE_train],axis=1)
    joblib.dump(encoder,"Cat_boost_step2_outer")
    
    AE_val=encoder.transform(X_val[list_var_to_encode].copy())
    AE_val.columns=[x+"_encoded" for x in AE_val.columns.tolist()]
    X_val_final=pd.concat([X_val,AE_val],axis=1)
    
    
    col_features=Features_to_bin+[x+"_encoded" for x in list_var_to_encode]+["Count_Pin","Count_Pin_Figure"]
    
    X_train_final=X_train_final.replace(np.nan,0)
    X_val_final=X_val_final.replace(np.nan,0)
    X_train_final=X_train_final[col_features].to_numpy()
    X_val_final=X_val_final[col_features].to_numpy()

    ratio=np.sum(y_train==0)/np.sum(y_train==1)
    


    xgb_cl = xgboost.XGBClassifier(objective="binary:logistic",n_estimators =150,max_depth = 10,scale_pos_weight=1)
    xgb_cl = xgboost.XGBClassifier(**Best_trial)

    xgb_cl.fit(X_train_final, y_train)
    joblib.dump(xgb_cl,"xgboost_step2_outter")

    res_train=xgb_cl.predict(X_train_final)
    res_test=xgb_cl.predict(X_val_final)

    X_train['Pred']=res_train
    X_val['Pred']=res_test
   
    
    y_train=X_train.groupby(["PanelID","FigureID","ComponentID"])['Target'].min()
    y_val=X_val.groupby(["PanelID","FigureID","ComponentID"])['Target'].min()


    res_train=X_train.groupby(["PanelID","FigureID","ComponentID"])['Pred'].min()
    res_test=X_val.groupby(["PanelID","FigureID","ComponentID"])['Pred'].min()




    print(confusion_matrix(y_train, res_train,labels=[0,1]))
    print(confusion_matrix(y_val,res_test,labels=[0,1]))

    print(f1_score(y_train, res_train,pos_label=0))
    print(f1_score(y_val, res_test,pos_label=0))

    f1_score_list.append(f1_score(y_val, res_test,pos_label=0))

0
[[ 164   84]
 [   1 8151]]
[[  28   11]
 [   0 1585]]
0.7941888619854721
0.835820895522388
1
[[ 164   84]
 [   1 8151]]
[[  14   13]
 [   1 1585]]
0.7941888619854721
0.6666666666666667
2
[[ 164   84]
 [   1 8151]]
[[  70   16]
 [   0 1664]]
0.7941888619854721
0.8974358974358974
3
[[ 164   84]
 [   1 8151]]
[[  30   28]
 [   0 1696]]
0.7941888619854721
0.6818181818181819
4
[[ 164   84]
 [   1 8151]]
[[  22   16]
 [   0 1621]]
0.7941888619854721
0.7333333333333334


In [34]:
np.mean(f1_score_list)

0.7630149949552936

/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-packages/optuna/samplers/_tpe/parzen_estimator.py:188: RuntimeWarning: divide by zero encountered in true_divide
  coefficient = 1 / z / p_accept
/home/phme/agaffet/.local/lib/python3.7/site-

[[ 356   95]
 [   1 8653]]


0.8439646064237593

In [32]:
356/(356+0.5*(95+1))

0.8811881188118812

['xgboost_step2_outter']

In [19]:
X_train_final.shape

(9105, 9)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
AE_train=encoder.fit_transform(X_train[list_var_to_encode].copy())


column_names=[x+"_encoded" for x in encoder.get_feature_names()]
dfint=pd.DataFrame(AE_train.toarray())
dfint.columns=column_names
X_train_final=pd.concat([X_train,dfint],axis=1)
joblib.dump(encoder,"One_hot_step2_outer")

AE_val=encoder.transform(X_val[list_var_to_encode].copy())
AE_val.columns=[x+"_encoded" for x in encoder.get_feature_names()]
dfint=pd.DataFrame(AE_val.toarray())
dfint.columns=column_names
X_val_final=pd.concat([X_val,dfint],axis=1)

In [31]:
Train_df.groupby('Target').size()

Target
0     451
1    8654
dtype: int64